<a href="https://colab.research.google.com/github/ShoaibSheriff/Computer-Vision/blob/master/Sparsenet_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''SparseNet models for Keras.
# Reference
- [Sparsely Connected Convolutional Networks](https://arxiv.org/abs/1801.05895)
- [Github](https://github.com/lyken17/sparsenet)
'''
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

import numpy as np
import warnings

from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.pooling import AveragePooling2D, MaxPooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.utils.layer_utils import convert_all_kernels_in_model, convert_dense_weights_data_format
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.applications.imagenet_utils import decode_predictions
import keras.backend as K


def preprocess_input(x, data_format=None):
    """Preprocesses a tensor encoding a batch of images.
    # Arguments
        x: input Numpy tensor, 4D.
        data_format: data format of the image tensor.
    # Returns
        Preprocessed tensor.
    """
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}

    if data_format == 'channels_first':
        if x.ndim == 3:
            # 'RGB'->'BGR'
            x = x[::-1, ...]
            # Zero-center by mean pixel
            x[0, :, :] -= 103.939
            x[1, :, :] -= 116.779
            x[2, :, :] -= 123.68
        else:
            x = x[:, ::-1, ...]
            x[:, 0, :, :] -= 103.939
            x[:, 1, :, :] -= 116.779
            x[:, 2, :, :] -= 123.68
    else:
        # 'RGB'->'BGR'
        x = x[..., ::-1]
        # Zero-center by mean pixel
        x[..., 0] -= 103.939
        x[..., 1] -= 116.779
        x[..., 2] -= 123.68

    x *= 0.017  # scale values

    return x


def SparseNet(input_shape=None, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=-1, nb_layers_per_block=-1,
              bottleneck=False, reduction=0.0, dropout_rate=0.0, weight_decay=1e-4, subsample_initial_block=False,
              include_top=True, weights=None, input_tensor=None,
              classes=10, activation='softmax'):
    '''Instantiate the SparseNet architecture,
        optionally loading weights pre-trained
        on CIFAR-10. Note that when using TensorFlow,
        for best performance you should set
        `image_data_format='channels_last'` in your Keras config
        at ~/.keras/keras.json.
        The model and the weights are compatible with both
        TensorFlow and Theano. The dimension ordering
        convention used by the model is the one
        specified in your Keras config file.
        # Arguments
            input_shape: optional shape tuple, only to be specified
                if `include_top` is False (otherwise the input shape
                has to be `(32, 32, 3)` (with `channels_last` dim ordering)
                or `(3, 32, 32)` (with `channels_first` dim ordering).
                It should have exactly 3 inputs channels,
                and width and height should be no smaller than 8.
                E.g. `(200, 200, 3)` would be one valid value.
            depth: number or layers in the DenseNet
            nb_dense_block: number of dense blocks to add to end (generally = 3)
            growth_rate: number of filters to add per dense block. Can be
                a single integer number or a list of numbers.
                If it is a list, length of list must match the length of
                `nb_layers_per_block`
            nb_filter: initial number of filters. -1 indicates initial
                number of filters is 2 * growth_rate
            nb_layers_per_block: number of layers in each dense block.
                Can be a -1, positive integer or a list.
                If -1, calculates nb_layer_per_block from the network depth.
                If positive integer, a set number of layers per dense block.
                If list, nb_layer is used as provided. Note that list size must
                be (nb_dense_block + 1)
            bottleneck: flag to add bottleneck blocks in between dense blocks
            reduction: reduction factor of transition blocks.
                Note : reduction value is inverted to compute compression.
            dropout_rate: dropout rate
            weight_decay: weight decay rate
            subsample_initial_block: Set to True to subsample the initial convolution and
                add a MaxPool2D before the dense blocks are added.
            include_top: whether to include the fully-connected
                layer at the top of the network.
            weights: one of `None` (random initialization) or
                'imagenet' (pre-training on ImageNet)..
            input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
                to use as image input for the model.
            classes: optional number of classes to classify images
                into, only to be specified if `include_top` is True, and
                if no `weights` argument is specified.
            activation: Type of activation at the top layer. Can be one of 'softmax' or 'sigmoid'.
                Note that if sigmoid is used, classes must be 1.
        # Returns
            A Keras model instance.
        '''

    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `cifar10` '
                         '(pre-training on CIFAR-10).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as ImageNet with `include_top`'
                         ' as true, `classes` should be 1000')

    if activation not in ['softmax', 'sigmoid']:
        raise ValueError('activation must be one of "softmax" or "sigmoid"')

    if activation == 'sigmoid' and classes != 1:
        raise ValueError('sigmoid activation can only be used when classes = 1')

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=32,
                                      min_size=8,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    x = _create_dense_net(classes, img_input, include_top, depth, nb_dense_block,
                          growth_rate, nb_filter, nb_layers_per_block, bottleneck, reduction,
                          dropout_rate, weight_decay, subsample_initial_block, activation)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='densenet')

    # load weights
    if weights == 'imagenet':
        weights_loaded = False

        if weights_loaded:
            if K.backend() == 'theano':
                convert_all_kernels_in_model(model)

            if K.image_data_format() == 'channels_first' and K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')

            print("Weights for the model were loaded successfully")

    return model


def _exponential_index_fetch(x_list):
    count = len(x_list)
    i = 1
    inputs = []
    while i <= count:
        inputs.append(x_list[count - i])
        i *= 2
    return inputs


def _conv_block(ip, nb_filter, bottleneck=False, dropout_rate=None, weight_decay=1e-4):
    ''' Apply BatchNorm, Relu, 3x3 Conv2D, optional bottleneck block and dropout
    Args:
        ip: Input keras tensor
        nb_filter: number of filters
        bottleneck: add bottleneck block
        dropout_rate: dropout rate
        weight_decay: weight decay factor
    Returns: keras tensor with batch_norm, relu and convolution2d added (optional bottleneck)
    '''
    concat_axis = 1 if K.image_data_format() == 'channels_first' else -1

    with K.name_scope('conv_block'):
        x = BatchNormalization(axis=concat_axis, momentum=0.1, epsilon=1e-5)(ip)
        x = Activation('relu')(x)

        if bottleneck:
            inter_channel = nb_filter * 4  # Obtained from https://github.com/liuzhuang13/DenseNet/blob/master/densenet.lua

            x = Conv2D(inter_channel, (1, 1), kernel_initializer='he_normal', padding='same', use_bias=False,
                       kernel_regularizer=l2(weight_decay))(x)
            x = BatchNormalization(axis=concat_axis, epsilon=1e-5, momentum=0.1)(x)
            x = Activation('relu')(x)

        x = Conv2D(nb_filter, (3, 3), kernel_initializer='he_normal', padding='same', use_bias=False)(x)
        if dropout_rate:
            x = Dropout(dropout_rate)(x)

    return x


def _dense_block(x, nb_layers, nb_filter, growth_rate, bottleneck=False, dropout_rate=None, weight_decay=1e-4,
                 grow_nb_filters=True, return_concat_list=False):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
    Args:
        x: keras tensor
        nb_layers: the number of layers of conv_block to append to the model.
        nb_filter: number of filters
        growth_rate: growth rate
        bottleneck: bottleneck block
        dropout_rate: dropout rate
        weight_decay: weight decay factor
        grow_nb_filters: flag to decide to allow number of filters to grow
        return_concat_list: return the list of feature maps along with the actual output
    Returns: keras tensor with nb_layers of conv_block appended
    '''
    concat_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x_list = [x]
    channel_list = [nb_filter]

    for i in range(nb_layers):
        #nb_channels = sum(_exponential_index_fetch(channel_list))

        x = _conv_block(x, growth_rate, bottleneck, dropout_rate, weight_decay)
        x_list.append(x)

        fetch_outputs = _exponential_index_fetch(x_list)
        x = concatenate(fetch_outputs, axis=concat_axis)

        channel_list.append(growth_rate)

    if grow_nb_filters:
        nb_filter = sum(_exponential_index_fetch(channel_list))

    if return_concat_list:
        return x, nb_filter, x_list
    else:
        return x, nb_filter


def _transition_block(ip, nb_filter, compression=1.0, weight_decay=1e-4):
    ''' Apply BatchNorm, Relu 1x1, Conv2D, optional compression, dropout and Maxpooling2D
    Args:
        ip: keras tensor
        nb_filter: number of filters
        compression: calculated as 1 - reduction. Reduces the number of feature maps
                    in the transition block.
        dropout_rate: dropout rate
        weight_decay: weight decay factor
    Returns: keras tensor, after applying batch_norm, relu-conv, dropout, maxpool
    '''
    concat_axis = 1 if K.image_data_format() == 'channels_first' else -1

    with K.name_scope('transition_block'):
        x = BatchNormalization(axis=concat_axis, epsilon=1e-5, momentum=0.1)(ip)
        x = Activation('relu')(x)
        x = Conv2D(int(nb_filter * compression), (1, 1), kernel_initializer='he_normal', padding='same', use_bias=False,
                   kernel_regularizer=l2(weight_decay))(x)
        x = AveragePooling2D((2, 2), strides=(2, 2))(x)

    return x


def _create_dense_net(nb_classes, img_input, include_top, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=-1,
                      nb_layers_per_block=-1, bottleneck=False, reduction=0.0, dropout_rate=None, weight_decay=1e-4,
                      subsample_initial_block=False, activation='softmax'):
    ''' Build the DenseNet model
    Args:
        nb_classes: number of classes
        img_input: tuple of shape (channels, rows, columns) or (rows, columns, channels)
        include_top: flag to include the final Dense layer
        depth: number or layers
        nb_dense_block: number of dense blocks to add to end (generally = 3)
        growth_rate: number of filters to add per dense block
        nb_filter: initial number of filters. Default -1 indicates initial number of filters is 2 * growth_rate
        nb_layers_per_block: number of layers in each dense block.
                Can be a -1, positive integer or a list.
                If -1, calculates nb_layer_per_block from the depth of the network.
                If positive integer, a set number of layers per dense block.
                If list, nb_layer is used as provided. Note that list size must
                be (nb_dense_block + 1)
        bottleneck: add bottleneck blocks
        reduction: reduction factor of transition blocks. Note : reduction value is inverted to compute compression
        dropout_rate: dropout rate
        weight_decay: weight decay rate
        subsample_initial_block: Set to True to subsample the initial convolution and
                add a MaxPool2D before the dense blocks are added.
        subsample_initial:
        activation: Type of activation at the top layer. Can be one of 'softmax' or 'sigmoid'.
                Note that if sigmoid is used, classes must be 1.
    Returns: keras tensor with nb_layers of conv_block appended
    '''

    concat_axis = 1 if K.image_data_format() == 'channels_first' else -1

    if reduction != 0.0:
        assert reduction <= 1.0 and reduction > 0.0, 'reduction value must lie between 0.0 and 1.0'

    # layers in each dense block
    if type(nb_layers_per_block) is list or type(nb_layers_per_block) is tuple:
        nb_layers = list(nb_layers_per_block)  # Convert tuple to list

        assert len(nb_layers) == (nb_dense_block), 'If list, nb_layer is used as provided. ' \
                                                   'Note that list size must be (nb_dense_block)'
        final_nb_layer = nb_layers[-1]
        nb_layers = nb_layers[:-1]
    else:
        if nb_layers_per_block == -1:
            assert (depth - 4) % 3 == 0, 'Depth must be 3 N + 4 if nb_layers_per_block == -1'
            count = int((depth - 4) / 3)

            if bottleneck:
                count = count // 2

            nb_layers = [count for _ in range(nb_dense_block)]
            final_nb_layer = count
        else:
            final_nb_layer = nb_layers_per_block
            nb_layers = [nb_layers_per_block] * nb_dense_block

    if type(growth_rate) is list or type(growth_rate) is tuple:
        growth_rate = list(growth_rate)
        assert len(growth_rate) == len(nb_layers)
    else:
        growth_rate = [growth_rate for _ in range(len(nb_layers))]

    # compute initial nb_filter if -1, else accept users initial nb_filter
    if nb_filter <= 0:
        nb_filter = 2 * growth_rate[0]

    # compute compression factor
    compression = 1.0 - reduction

    # Initial convolution
    if subsample_initial_block:
        initial_kernel = (7, 7)
        initial_strides = (2, 2)
    else:
        initial_kernel = (3, 3)
        initial_strides = (1, 1)

    x = Conv2D(nb_filter, initial_kernel, kernel_initializer='he_normal', padding='same',
               strides=initial_strides, use_bias=False, kernel_regularizer=l2(weight_decay))(img_input)

    if subsample_initial_block:
        x = BatchNormalization(axis=concat_axis, epsilon=1e-5, momentum=0.1)(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = _dense_block(x, nb_layers[block_idx], nb_filter, growth_rate[block_idx], bottleneck=bottleneck,
                                    dropout_rate=dropout_rate, weight_decay=weight_decay)
        # add transition_block
        x = _transition_block(x, nb_filter, compression=compression, weight_decay=weight_decay)
        nb_filter = int(nb_filter * compression)

    # The last dense_block does not have a transition_block
    x, nb_filter = _dense_block(x, final_nb_layer, nb_filter, growth_rate[-1], bottleneck=bottleneck,
                                dropout_rate=dropout_rate, weight_decay=weight_decay)

    x = BatchNormalization(axis=concat_axis, epsilon=1e-5, momentum=0.1)(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)

    if include_top:
        x = Dense(nb_classes, activation=activation)(x)

    return x


Using TensorFlow backend.


In [0]:
from __future__ import print_function

import os.path

import numpy as np
import sklearn.metrics as metrics

from keras.datasets import cifar10
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

batch_size = 100
nb_classes = 10
nb_epoch = 100

img_rows, img_cols = 32, 32
img_channels = 3

img_dim = (img_channels, img_rows, img_cols) if K.image_dim_ordering() == "th" else (img_rows, img_cols, img_channels)
depth = 40
nb_dense_block = 3
growth_rate = 24
nb_filter = -1
dropout_rate = 0.0  # 0.0 for data augmentation

model = SparseNet(img_dim, classes=nb_classes, depth=depth, nb_dense_block=nb_dense_block,
                            growth_rate=growth_rate, nb_filter=nb_filter, dropout_rate=dropout_rate, weights=None)
print("Model created")

model.summary()
optimizer = Adam(lr=5e-3, amsgrad=True)  # Using Adam instead of SGD to speed up training
#optimizer = SGD(lr=0.1, decay=0.0001, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])
print("Finished compiling")
print("Building model...")

(trainX, trainY), (testX, testY) = cifar10.load_data()

trainX = trainX.astype('float32')
testX = testX.astype('float32')

trainX = preprocess_input(trainX)
testX = preprocess_input(testX)

cifar_mean = trainX.mean(axis=(0, 1, 2), keepdims=True)
cifar_std = trainX.std(axis=(0, 1, 2), keepdims=True)

trainX = (trainX - cifar_mean) / (cifar_std + 1e-8)
testX = (testX - cifar_mean) / (cifar_std + 1e-8)

Y_train = np_utils.to_categorical(trainY, nb_classes)
Y_test = np_utils.to_categorical(testY, nb_classes)

generator = ImageDataGenerator(width_shift_range=8. / 32,
                               height_shift_range=8. / 32,
                               horizontal_flip=True)

generator.fit(trainX, seed=0)


Model created
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 48)   1296        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 48)   192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 48)   0           batch_normalization_1[0][0]      
_______________________________________________________________________________________________

170500096/170498071 [==============================] - 35s 0us/step


In [0]:
model_file = "SparseNet-40-24-CIFAR10.h5"

lr_reducer = ReduceLROnPlateau(monitor='val_acc', factor=0.5,
                               cooldown=0, patience=4, min_lr=1e-9, verbose = 1)
model_checkpoint = ModelCheckpoint(model_file, monitor="val_acc", save_best_only=True, verbose=1)

callbacks = [model_checkpoint, lr_reducer]

model.fit_generator(generator.flow(trainX, Y_train, batch_size=batch_size),
                    steps_per_epoch=len(trainX) // batch_size, epochs=30,
                    callbacks=callbacks,
                    validation_data=(testX, Y_test),
                    validation_steps=testX.shape[0] // batch_size, verbose=1)

from keras.models import load_model
modelB = load_model(model_file)

''''yPreds = modelB.predict(testX)
yPred = np.argmax(yPreds, axis=1)
yTrue = testY

accuracy = metrics.accuracy_score(yTrue, yPred) * 100
error = 100 - accuracy
print("Accuracy : ", accuracy)
print("Error : ", error)'''


modelB.save("till30.h5")
from google.colab import files
files.download("till30.h5")

Epoch 1/30
500/500 [==============================] - 236s 472ms/step - loss: 1.6998 - acc: 0.3799 - val_loss: 1.4386 - val_acc: 0.4874

Epoch 00001: val_acc improved from -inf to 0.48740, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 2/30
165/500 [========>.....................] - ETA: 2:21 - loss: 1.3532 - acc: 0.5182

500/500 [==============================] - 225s 451ms/step - loss: 1.2431 - acc: 0.5621 - val_loss: 1.3373 - val_acc: 0.5771

Epoch 00002: val_acc improved from 0.48740 to 0.57710, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 3/30
229/500 [============>.................] - ETA: 1:54 - loss: 1.0562 - acc: 0.6348

500/500 [==============================] - 225s 450ms/step - loss: 1.0156 - acc: 0.6494 - val_loss: 0.9813 - val_acc: 0.6731

Epoch 00003: val_acc improved from 0.57710 to 0.67310, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 4/30
245/500 [=============>................] - ETA: 1:47 - loss: 0.8918 - acc: 0.6971

500/500 [==============================] - 225s 450ms/step - loss: 0.8661 - acc: 0.7058 - val_loss: 0.9143 - val_acc: 0.6981

Epoch 00004: val_acc improved from 0.67310 to 0.69810, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 5/30
249/500 [=============>................] - ETA: 1:45 - loss: 0.7748 - acc: 0.7378

500/500 [==============================] - 225s 450ms/step - loss: 0.7698 - acc: 0.7407 - val_loss: 0.9296 - val_acc: 0.7126

Epoch 00005: val_acc improved from 0.69810 to 0.71260, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 6/30
250/500 [==============>...............] - ETA: 1:45 - loss: 0.7067 - acc: 0.7645

500/500 [==============================] - 224s 449ms/step - loss: 0.6878 - acc: 0.7720 - val_loss: 0.7012 - val_acc: 0.7699

Epoch 00006: val_acc improved from 0.71260 to 0.76990, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 7/30
251/500 [==============>...............] - ETA: 1:44 - loss: 0.6439 - acc: 0.7871

500/500 [==============================] - 224s 449ms/step - loss: 0.6361 - acc: 0.7900 - val_loss: 0.6803 - val_acc: 0.7817

Epoch 00007: val_acc improved from 0.76990 to 0.78170, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 8/30
251/500 [==============>...............] - ETA: 1:44 - loss: 0.5888 - acc: 0.8069

500/500 [==============================] - 225s 450ms/step - loss: 0.5873 - acc: 0.8084 - val_loss: 0.6059 - val_acc: 0.8087

Epoch 00008: val_acc improved from 0.78170 to 0.80870, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 9/30
251/500 [==============>...............] - ETA: 1:45 - loss: 0.5510 - acc: 0.8229

500/500 [==============================] - 225s 451ms/step - loss: 0.5507 - acc: 0.8229 - val_loss: 0.6663 - val_acc: 0.7995

Epoch 00009: val_acc did not improve from 0.80870
Epoch 10/30
296/500 [================>.............] - ETA: 1:26 - loss: 0.5295 - acc: 0.8289

500/500 [==============================] - 225s 450ms/step - loss: 0.5220 - acc: 0.8315 - val_loss: 0.5627 - val_acc: 0.8277

Epoch 00010: val_acc improved from 0.80870 to 0.82770, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 11/30
256/500 [==============>...............] - ETA: 1:42 - loss: 0.5007 - acc: 0.8402

In [0]:
modelB.save("till30.h5")
from google.colab import files
files.download("till30.h5")

In [0]:
model.fit_generator(generator.flow(trainX, Y_train, batch_size=batch_size),
                    steps_per_epoch=len(trainX) // batch_size, epochs=30,
                    callbacks=callbacks,
                    validation_data=(testX, Y_test),
                    validation_steps=testX.shape[0] // batch_size, verbose=1)

from keras.models import load_model
modelB = load_model(model_file)

''''yPreds = modelB.predict(testX)
yPred = np.argmax(yPreds, axis=1)
yTrue = testY

accuracy = metrics.accuracy_score(yTrue, yPred) * 100
error = 100 - accuracy
print("Accuracy : ", accuracy)
print("Error : ", error)'''


modelB.save("30to60.h5")
from google.colab import files
files.download("30to60.h5")

Epoch 1/30
500/500 [==============================] - 217s 434ms/step - loss: 0.2701 - acc: 0.9165 - val_loss: 0.4179 - val_acc: 0.8804

Epoch 00001: val_acc did not improve from 0.88780
Epoch 2/30
193/500 [==========>...................] - ETA: 2:04 - loss: 0.2476 - acc: 0.9222

500/500 [==============================] - 217s 433ms/step - loss: 0.2591 - acc: 0.9192 - val_loss: 0.4055 - val_acc: 0.8891

Epoch 00002: val_acc improved from 0.88780 to 0.88910, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 3/30
236/500 [=============>................] - ETA: 1:47 - loss: 0.2504 - acc: 0.9217

500/500 [==============================] - 217s 434ms/step - loss: 0.2554 - acc: 0.9204 - val_loss: 0.3711 - val_acc: 0.8932

Epoch 00003: val_acc improved from 0.88910 to 0.89320, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 4/30
247/500 [=============>................] - ETA: 1:43 - loss: 0.2499 - acc: 0.9222

500/500 [==============================] - 217s 435ms/step - loss: 0.2566 - acc: 0.9202 - val_loss: 0.3680 - val_acc: 0.8973

Epoch 00004: val_acc improved from 0.89320 to 0.89730, saving model to SparseNet-40-24-CIFAR10.h5
Epoch 5/30
250/500 [==============>...............] - ETA: 1:40 - loss: 0.2438 - acc: 0.9251

500/500 [==============================] - 216s 433ms/step - loss: 0.2486 - acc: 0.9228 - val_loss: 0.4353 - val_acc: 0.8807

Epoch 00005: val_acc did not improve from 0.89730
Epoch 6/30
297/500 [================>.............] - ETA: 1:22 - loss: 0.2475 - acc: 0.9230

...

500/500 [==============================] - 222s 443ms/step - loss: 0.1084 - acc: 0.9701 - val_loss: 0.3622 - val_acc: 0.9134

Epoch 00027: val_acc did not improve from 0.92270
Epoch 28/30
310/500 [=================>............] - ETA: 1:19 - loss: 0.1057 - acc: 0.9704

500/500 [==============================] - 223s 445ms/step - loss: 0.1061 - acc: 0.9706 - val_loss: 0.3916 - val_acc: 0.9092

Epoch 00028: val_acc did not improve from 0.92270
Epoch 29/30
313/500 [=================>............] - ETA: 1:18 - loss: 0.1029 - acc: 0.9715

500/500 [==============================] - 223s 446ms/step - loss: 0.1048 - acc: 0.9708 - val_loss: 0.3538 - val_acc: 0.9139

Epoch 00029: val_acc did not improve from 0.92270
Epoch 30/30
314/500 [=================>............] - ETA: 1:16 - loss: 0.1037 - acc: 0.9717

500/500 [==============================] - 223s 445ms/step - loss: 0.1039 - acc: 0.9715 - val_loss: 0.3635 - val_acc: 0.9157

Epoch 00030: val_acc did not improve from 0.92270

Epoch 00030: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.


MessageError: ignored

In [0]:
modelB.save("30to60.h5")
from google.colab import files
files.download("30to60.h5")

In [0]:
model.fit_generator(generator.flow(trainX, Y_train, batch_size=batch_size),
                    steps_per_epoch=len(trainX) // batch_size, epochs=30,
                    callbacks=callbacks,
                    validation_data=(testX, Y_test),
                    validation_steps=testX.shape[0] // batch_size, verbose=1)

from keras.models import load_model
modelB = load_model(model_file)

''''yPreds = modelB.predict(testX)
yPred = np.argmax(yPreds, axis=1)
yTrue = testY

accuracy = metrics.accuracy_score(yTrue, yPred) * 100
error = 100 - accuracy
print("Accuracy : ", accuracy)
print("Error : ", error)'''


modelB.save("60to90.h5")
from google.colab import files
files.download("60to90.h5")

Epoch 1/30
500/500 [==============================] - 222s 443ms/step - loss: 0.0960 - acc: 0.9746 - val_loss: 0.3557 - val_acc: 0.9159

Epoch 00001: val_acc did not improve from 0.92270
Epoch 2/30
193/500 [==========>...................] - ETA: 2:07 - loss: 0.0902 - acc: 0.9766

500/500 [==============================] - 222s 443ms/step - loss: 0.0914 - acc: 0.9761 - val_loss: 0.3483 - val_acc: 0.9188

Epoch 00002: val_acc did not improve from 0.92270
Epoch 3/30
279/500 [===============>..............] - ETA: 1:32 - loss: 0.0889 - acc: 0.9781

500/500 [==============================] - 223s 446ms/step - loss: 0.0909 - acc: 0.9771 - val_loss: 0.3630 - val_acc: 0.9143

Epoch 00003: val_acc did not improve from 0.92270
Epoch 4/30
305/500 [=================>............] - ETA: 1:22 - loss: 0.0883 - acc: 0.9768

500/500 [==============================] - 224s 448ms/step - loss: 0.0883 - acc: 0.9772 - val_loss: 0.3687 - val_acc: 0.9136

Epoch 00004: val_acc did not improve from 0.92270
Epoch 5/30
313/500 [=================>............] - ETA: 1:17 - loss: 0.0891 - acc: 0.9765

500/500 [==============================] - 224s 447ms/step - loss: 0.0893 - acc: 0.9762 - val_loss: 0.3796 - val_acc: 0.9163

Epoch 00005: val_acc did not improve from 0.92270
Epoch 6/30
316/500 [=================>............] - ETA: 1:17 - loss: 0.0882 - acc: 0.9771

...

500/500 [==============================] - 223s 445ms/step - loss: 0.0722 - acc: 0.9824 - val_loss: 0.3624 - val_acc: 0.9184

Epoch 00027: val_acc did not improve from 0.92490
Epoch 28/30
308/500 [=================>............] - ETA: 1:19 - loss: 0.0720 - acc: 0.9826

500/500 [==============================] - 222s 443ms/step - loss: 0.0714 - acc: 0.9826 - val_loss: 0.3543 - val_acc: 0.9206

Epoch 00028: val_acc did not improve from 0.92490
Epoch 29/30
313/500 [=================>............] - ETA: 1:17 - loss: 0.0727 - acc: 0.9825

500/500 [==============================] - 224s 449ms/step - loss: 0.0730 - acc: 0.9824 - val_loss: 0.3782 - val_acc: 0.9186

Epoch 00029: val_acc did not improve from 0.92490
Epoch 30/30
314/500 [=================>............] - ETA: 1:17 - loss: 0.0729 - acc: 0.9820

500/500 [==============================] - 221s 443ms/step - loss: 0.0708 - acc: 0.9825 - val_loss: 0.3351 - val_acc: 0.9240

Epoch 00030: val_acc did not improve from 0.92490

Epoch 00030: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.


MessageError: ignored